## Vitaly Kozhemyak [nickname: Vitaliy Kozhemyak(OM)]

## Text preprocessing

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install pymorphy2

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 8.2MB 7.6MB/s 


In [1]:
import re
import nltk
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

from functools import lru_cache
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []
    
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=MorphAnalyzer()):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
# !unzip -q lenta-ru-ozon-2020.zip -d .

In [6]:
data_path = "/content/drive/My Drive/Classroom/Natural Language Processing/data/"

In [7]:
cd /content/drive/My Drive/Classroom/Natural Language Processing/data/

/content/drive/My Drive/Classroom/Natural Language Processing/data


## Load train-test data

In [26]:
train_data = pd.read_csv("/content/drive/My Drive/Classroom/Natural Language Processing/data/lenta-ru-train.csv")
test_data = pd.read_csv("/content/drive/My Drive/Classroom/Natural Language Processing/data/lenta-ru-test.csv")

## Extract lemmas from clean texts

In [ ]:
with Pool() as p:
    lemmas = list(tqdm(p.imap(clean_text, train_data['text']), total=len(train_data)))
    title_lemmas = list(tqdm(p.imap(clean_text, train_data['title']), total=len(train_data)))
    
train_data['lemmas'] = lemmas
train_data['title_lemmas'] = title_lemmas
train_data.sample(5)

In [ ]:
train_data.head()

,Unnamed: 0,title,text,topic,topic_label,lemmas,title_lemmas
0,0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0,глава росстат владимир соколиный заявить январ...,инфляция январь составить процент
1,1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3,российский кинематографист появиться новый про...,никита михалков учредить день российский кино
2,2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1,медицинский штаб миланский интер обнародовать ...,марко матерацци вернуться строй матч цска
3,3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3,премия книга номинация вручить московский межд...,определить лауреат премия книга
4,4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1,сборная португалия сыграть вничью команда испа...,роналда штрафной спасти португалец поражение м...


In [ ]:
train_data.to_csv("/content/drive/My Drive/Classroom/Natural Language Processing/data/good_train_data.csv")

In [ ]:
with Pool() as p:
    lemmas = list(tqdm(p.imap(clean_text, test_data['text']), total=len(test_data)))
    
test_data['lemmas'] = lemmas
test_data.sample(5)

100%|██████████| 51660/51660 [15:34<00:00, 55.31it/s]


,title,text,lemmas
37357,Глава Крыма рассказал о превысивших лучшие вре...,"Крым уже вышел на показатели, превышающие все ...",крым выйти показатель превышать хороший состав...
43753,Ивана Дыховичного похоронят на Новодевичьем кл...,"Актер и режиссер Иван Дыховичный, скончавшийся...",актёр режиссёр иван дыховичный скончаться тяжё...
40978,"Михаэль Баллак пропустит матч с ""Барселоной""",Бывший капитан сборной Германии по футболу Мих...,бывший капитан сборная германия футбол михаэль...
46790,Запершего судей в раздевалке президента «Трабз...,Федерация футбола Турции дисквалифицировала пр...,федерация футбол турция дисквалифицировать пре...
44068,СМИ узнали подробности обучения в секретном «у...,"Тренинговая программа для сотрудников Apple, б...",тренинговый программа сотрудник apple известны...


In [ ]:
test_data.to_csv("/content/drive/My Drive/Classroom/Natural Language Processing/data/good_test_data.csv")

In [4]:
train_data = pd.read_csv("/content/drive/My Drive/Classroom/Natural Language Processing/data/good_train_data.csv")
test_data = pd.read_csv("/content/drive/My Drive/Classroom/Natural Language Processing/data/good_test_data.csv")

In [5]:
train_data.dropna(inplace=True)
test_data = test_data.fillna(test_data.lemmas[0])

# Fasttext classification

In [9]:
with open('clean_text.txt', 'w') as f:
    for s, label, title in zip(train_data["lemmas"], 
                                train_data["topic_label"],
                                train_data["title_lemmas"]):
        f.write(f"__label__{label} " + s)
        f.write('\n')
        f.write(f"__label__{label} " + title)
        f.write('\n')

In [ ]:
import fasttext

model = fasttext.train_supervised(data_path + "clean_text.txt")

In [17]:
answer = []

for s in test_data["lemmas"]:
    answer.append(int(model.predict(s)[0][0][-1]))

In [18]:
res_df = pd.DataFrame()
res_df["id"] = test_data.index
res_df['category'] = answer
res_df[['id','category']].to_csv('dummy_submission.csv', index=False)

# Embeddingns

In [9]:
with open('clean_text_without_labels.txt', 'w') as f:
    for s, title in zip(train_data["lemmas"],
                               train_data["title_lemmas"]):
        f.write(s)
        f.write('\n')
        f.write(title)
        f.write('\n')

In [10]:
import fasttext

model = fasttext.train_unsupervised(data_path + "clean_text_without_labels.txt", model='cbow')

In [12]:
model.save_model("model_fasttext.bin")

In [18]:
import numpy as np

def mean_embedding(text):
    tmp = np.zeros(100)
    cnt = 0

    w = text.split(" ")
    for word in model.words:
        if word in w:
            tmp += model[word]
            cnt += 1
    return tmp / cnt


with Pool() as p:
    arr = list(tqdm(p.imap(mean_embedding, test_data['lemmas']), total=test_data.shape[0]))

100%|██████████| 51660/51660 [1:38:32<00:00,  8.74it/s]


In [19]:
W_fast = np.zeros((test_data.shape[0], 100))
for i in range(test_data.shape[0]):
    W_fast[i, :] = arr[i]
np.save("/content/drive/My Drive/Classroom/Natural Language Processing/data/W_fast.npy", W_fast)

In [26]:
n = 10000
with Pool() as p:
    arr = list(tqdm(p.imap(mean_embedding, train_data['lemmas'][:n]), total=n))

100%|██████████| 10000/10000 [19:04<00:00,  8.74it/s]


In [27]:
W = np.zeros((n, 100))
for i in range(n):
    W[i, :] = arr[i]
np.save("/content/drive/My Drive/Classroom/Natural Language Processing/data/W.npy", W)

In [28]:
from sklearn.linear_model import LogisticRegression 

clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(W, train_data["topic_label"][:n])
pred = clf.predict(W_fast)

In [29]:
res_df = pd.DataFrame()
res_df["id"] = test_data.index
res_df['category'] = pred
res_df[['id','category']].to_csv('dummy_submission.csv', index=False)

# Tf-Idf model

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_data[["lemmas", "title_lemmas"]], train_data.topic_label)

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(train_data["lemmas"]) 

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(bow, train_data["topic_label"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
test_data = test_data.fillna(test_data.lemmas[0])

In [ ]:
pred = clf.predict(vec.transform(test_data["lemmas"]))

In [ ]:
accuracy_score(y_test, pred)

0.916

In [ ]:
cd /content/drive/My Drive/Classroom/Natural Language Processing/data

/content/drive/My Drive/Classroom/Natural Language Processing/data


In [ ]:
pred = clf.predict(vec.transform(test_data.lemmas))

In [ ]:
res_df = pd.DataFrame()
res_df["id"] = test_data.index
res_df['category'] = pred
res_df[['id','category']].to_csv('dummy_submission.csv', index=False)